<a href="https://colab.research.google.com/github/sebavassou/SIG/blob/main/ambiente_sig_ciencia_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalar as bibliotecas de SIG
*Ambiente Linux*
1.   Atualiza o repositório local (sincroniza) e depois o sistema  - ***'apt'***
2.   Instala as dependências para as bibliotecas Python - ***'apt'***
3.   Atualiza o ***'pip'*** (instalado das bibliotecas python)
4.   Instala as bibliotecas de Python para SIG

Obs.:
- O Google Colab já possui alguma das bibliotecas
- Algumas bibliotecas depende de outras (instala automáticamente)

In [ ]:
# prompt: instalar as bibliotecas: GeoPandas, GDAL, Shapely, RasterIO, Xarray e ipyleaflet

!apt update
!apt upgrade
!apt install libspatialindex-dev
!apt install libgdal-dev libgdal-pcraster-dev libgdal-perl
!pip install --upgrade pip
!pip install shapely --no-binary shapely
!pip install geopandas
!pip install pycrs
!pip install rasterio
!pip install xarray
!pip install ipyleaflet

## Testar as bibliotecas

In [ ]:
import geopandas as gpd
from osgeo import ogr, gdal, osr

from shapely.geometry import Point, LineString, Polygon
import rasterio
import xarray as xr
from ipyleaflet import Map, Marker, GeoJSON

## Testando ambiente do Colab

In [ ]:
#!pwd
#!ls
#!cat /etc/os-release

## GDAL

### Raster (imagens)

In [ ]:
# Baixar imagem via wget
!wget 'https://github.com/mommermi/geotiff_sample/raw/refs/heads/master/sample.tif'

In [ ]:
gdal.DontUseExceptions()
ds = gdal.Open('sample.tif')

msg = f"""
Total de bandas : {ds.RasterCount}
Tamanho da imagem : {ds.RasterXSize} x {ds.RasterYSize}
"""
print( msg )

# Dataset (GDAL) p/ Array Numpy
array_ds = ds.ReadAsArray()
msg = ''
lin, col = 0, 1
for band in range(len(array_ds)):
    msg += f"Banda: {band} - Linha {lin} - Coluna {col} = {array_ds[ band, lin, col]}\n"
print( msg )

ds = None

### Vetor

In [ ]:
url = 'WFS:https://panorama.sipam.gov.br/geoserver/publico/ows'
ds = ogr.Open( url )

for id in range( ds.GetLayerCount() ):
    layer = ds.GetLayerByIndex( id )
    msg = f"{id}) Descrição: {layer.GetDescription()}"
    msg += f"\n- Total de feições: {layer.GetFeatureCount()}"
    msg += f"\n- Tipo de Geometria: {ogr.GeometryTypeToName( ( layer.GetGeomType() ) )}"
    print( msg )

layer = ds.GetLayerByIndex( 8 ) # publico:ponte
feature = layer.GetNextFeature()
print( f"\n\n{feature}")
ds = None

## Shapely

In [ ]:
from shapely.geometry import Polygon

poligono1 = Polygon([(0, 0), (2, 0), (2, 2), (0, 2)])
poligono2 = Polygon([(1, 1), (3, 1), (3, 3), (1, 3)])

In [ ]:
result = poligono1.intersection( poligono2 )
print(result.wkt)
result

In [ ]:
result = poligono1.difference(poligono2)
print(result.wkt)
result

## RasterIO

In [ ]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np

def stretch_contrast(band):
    # Contraste Cumulativo (2-98%)
    p2, p98 = np.percentile(band, (2, 98))
    return np.clip((band - p2) / (p98 - p2), 0, 1)

with rasterio.open('sample.tif') as src:
    print("Dimensões:", src.width, "x", src.height)
    print("Número de bandas:", src.count)

    r, g, b = src.read(1), src.read(2), src.read(3)

    rgb = np.dstack((r, g, b)).astype(np.float32)
    rgb /= rgb.max()
    #rgb_stretched = np.dstack([stretch_contrast(rgb[:, :, i]) for i in range(3)])

    # Exibindo a banda
    plt.figure(figsize=(10, 10))
    plt.imshow(rgb)
    #plt.imshow(rgb_stretched)
    plt.title('Imagem de Satélite (RGB)')
    plt.axis('off')
    plt.show()

In [ ]:
rgb = np.dstack((r, g, b))
rgb[:,:,0] # Todas as Linhas, Todas Colunas, Celula (0/R, 1/G, 2/B)

In [ ]:
rgb[0, 1, :] # 1a Linha, 2a Coluna, Todas as células (RGB)

## Geopandas

In [ ]:
# Baixar imagem via wget
!wget https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/diario/Brasil/focos_diario_br_20250618.csv

In [ ]:
!head focos_diario_br_20250618.csv

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

filepath = 'focos_diario_br_20250618.csv'

df = pd.read_csv( filepath )

geometry = [ Point(xy) for xy in zip(df['lon'], df['lat']) ]
df.drop(['lon', 'lat'], axis=1, inplace=True)

# Convertendo para GeoDataFrame
gdf_focos = gpd.GeoDataFrame(df, geometry=geometry)

df = None

# Definindo o sistema de referência (WGS84)
gdf_focos.set_crs(epsg=4326, inplace=True)

# Exibindo os primeiros registros
gdf_focos.head()

In [ ]:
geojson = """
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -56.58792582779658,
              -9.542233284913337
            ],
            [
              -56.58792582779658,
              -11.23479269385929
            ],
            [
              -54.836755873173786,
              -11.23479269385929
            ],
            [
              -54.836755873173786,
              -9.542233284913337
            ],
            [
              -56.58792582779658,
              -9.542233284913337
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}
"""


In [ ]:
import json

# ROI
json.loads(geojson)["features"]
gdf_roi = gpd.GeoDataFrame.from_features( json.loads(geojson)["features"] )
gdf_roi.set_crs(epsg=4326, inplace=True)

# Filter
gdf_filter = gdf_focos[ gdf_focos.geometry.within( gdf_roi.unary_union ) ]
gdf_filter.head()

## Mapa

In [ ]:
from ipyleaflet import Map, GeoData, Popup #,LayerControl
from ipywidgets import HTML, Layout
from shapely.geometry import shape

def on_click(event, feature, **kwargs):
    props = feature['properties']
    geom = shape(feature['geometry'])
    coords = [geom.y, geom.x]
    #html_content = "<b>Atributos:</b><br>" + "<br>".join([f"{k}: {v}" for k, v in props.items()])
    html_content = """
        <div style="font-family: Arial; font-size: 13px; color: #333;">
            <b style="font-size: 14px;">Atributos:</b><br>
            {}
        </div>
        """.format("<br>".join([f"<b>{k}</b>: {v}" for k, v in props.items()]))

    popup = Popup(
        location=coords,
        child=HTML(value=html_content),
        close_button=True,
        auto_close=False,
        close_on_escape_key=False
    )
    m.popups = []  # Remove popups anteriores
    m.add(popup)


center = [-9.45, -56.58] # Lat, Long
m = Map(center=center, zoom=6, layout=Layout(height="500px"))

layer = GeoData(
    geo_dataframe=gdf_filter,
    name = 'Focos',
    name_property='id'
)
layer.on_click(on_click)

m.add_layer(layer)
#m.add_control(LayerControl())
m